In [ ]:
library(future)

supported_strategies <- future:::supportedStrategies()
print(supported_strategies)  
plan("multicore", workers = 10)    
options(future.globals.maxSize = 60 * 1024^3)  # 256GB

current_plan <- plan()
print(current_plan)


library(Seurat)
library(stringr)


seob_list <- list() 

phase = 'TM0' 
load(file = str_c('/data/users/yuantingting/yuantingting_fd18e4ec5adc46dd89e2e5c607e9e7e3/online/output/Seurat/Merge/', phase, '/', phase, '_merged_all.rdata')) 

seob_list[[1]] <- seob

phase = 'TM1' 
load(file = str_c('/data/users/yuantingting/yuantingting_fd18e4ec5adc46dd89e2e5c607e9e7e3/online/output/Seurat/Merge/', phase, '/', phase, '_merged_all.rdata')) 

seob_list[[2]] <- seob

phase = 'TM2' 
load(file = str_c('/data/users/yuantingting/yuantingting_fd18e4ec5adc46dd89e2e5c607e9e7e3/online/output/Seurat/Merge/', phase, '/', phase, '_merged_all.rdata')) 
seob_list[[3]] <- seob

phase = 'LTM' 
load(file = str_c('/data/users/yuantingting/yuantingting_fd18e4ec5adc46dd89e2e5c607e9e7e3/online/output/Seurat/Merge/', phase, '/', phase, '_merged_all.rdata')) 
seob_list[[4]] <- seob

phase = 'EFM' 
load(file = str_c('/data/users/yuantingting/yuantingting_fd18e4ec5adc46dd89e2e5c607e9e7e3/online/output/Seurat/Merge/', phase, '/', phase, '_merged_all.rdata')) 
seob_list[[5]] <- seob

phase = 'SIM' 
load(file = str_c('/data/users/yuantingting/yuantingting_fd18e4ec5adc46dd89e2e5c607e9e7e3/online/output/Seurat/Merge/', phase, '/', phase, '_merged_all.rdata')) 
seob@meta.data$sample <- "FM"
seob_list[[6]] <- seob

outdir = paste0('/data/work/output/Seurat/6phase_SeuratV5_test1/')

dir.create(outdir, recursive = TRUE, showWarnings = FALSE)


seob_list <- lapply(seob_list, function(obj) {
  obj[["RNA"]] <- as(obj[["RNA"]], "Assay5")  
  DefaultAssay(obj) <- "RNA"                 
  return(obj)
})

merged_obj <- merge(x = seob_list[[1]], y = list(seob_list[[2]], seob_list[[3]], seob_list[[4]], seob_list[[5]], seob_list[[6]]))

merged_obj <- SCTransform(object = merged_obj) 

merged_obj <- RunPCA(object = merged_obj)

merged_obj

merged_obj <- IntegrateLayers(object = merged_obj,normalization.method = "SCT", method = RPCAIntegration, orig.reduction = "pca", new.reduction = "integrated.rpca",
    verbose = FALSE)

merged_obj <- FindNeighbors(object = merged_obj, dims = 1:30) %>%
    FindClusters(future.seed=TRUE) %>%
    RunUMAP(dims = 1:30)

# now that integration is complete, rejoin layers
merged_obj[["RNA"]] <- JoinLayers(merged_obj)

# Visualization
p1 <- DimPlot(merged_obj, reduction = "umap", group.by = "sample")

mycolor <- c(
  '#E41A1C', '#377EB8', '#4DAF4A', '#984EA3', '#FF7F00',  
  '#FFFF33', '#A65628', '#F781BF', '#66C2A5', '#FC8D62',  
  '#8DA0CB', '#E78AC3', '#A6D854', '#FFD92F', '#E5C494',   
  '#B3B3B3', '#1B9E77', '#D95F02', '#7570B3', '#E7298A',   
  '#66A61E', '#E6AB02', '#A6761D', '#666666', '#A6CEE3',   
  '#1F78B4', '#B2DF8A', '#33A022', '#8C564B', '#17BECF',   
  '#BCBD22', '#9467BD'                                      
)


sample = '6phase'
pdf(paste0(outdir, "Batcheffect_umap_SCT",sample,".pdf"), height = 8, width = 10)

DimPlot(
  merged_obj,
  cols = mycolor,
  reduction = "umap",
  group.by = "sample"
) 

dev.off()

pdf(str_c(outdir, "Cluster_",sample, ".pdf"), height = 8, width = 16)

p1 <- DimPlot(
    merged_obj,
    cols = mycolor,
    reduction = "pca",
    raster=FALSE,
    group.by = "seurat_clusters",
    label = T
) 
p2 <- DimPlot(
  merged_obj,
  reduction = "tsne",
  group.by = "seurat_clusters",
  label = T
) 
p3 <- DimPlot(
    merged_obj,
    cols = mycolor,
    reduction = "umap",
    raster=FALSE,
    group.by = "seurat_clusters",
    label = T
) 

p1+p2+p3

dev.off()

merged_obj$sample <- factor(merged_obj$sample, levels = c('TM0','TM1','TM2','LTM','EFM','FM'))

pdf(str_c(outdir, "Cluster_splited_",sample, ".pdf"), height = 4, width = 24)

DimPlot(
    merged_obj,
    cols = mycolor,
    reduction = "umap",
    split.by = "sample",
    raster=FALSE,
    group.by = "seurat_clusters",
    label = T
) 

dev.off()


save(merged_obj,file = str_c(outdir, "36clusters_merged_",sample,".rdata"))

saveRDS(merged_obj,file = str_c(outdir, "36clusters_",sample, ".rds"))